# Подключения и функции

In [2]:
# Импорт

import pandas as pd
from clickhouse_driver import Client
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Функция создания датафрейма

def create_dataframe(user, password, table_name='films'):
    client = Client('90.156.216.60', user=user, password=password, database='it_minimalist_CH')
    query = f'SELECT * FROM {table_name}'
    df = client.query_dataframe(query)
    return df

В ячейке ниже создается датафрейм в переменной `df`.
<br>

- Замените `'your_username'`, `'your_password'` на ваши учетные данные, которые получили на почту (у каждого своя учетная запись с индивидуальным доступом к БД Clickhouse).
<br>
- Как вы заметили в параметрах функции `create_dataframe`, по умолчанию датафрейм создатеся из таблицы `'films'`. Также в базе данных лежит таблица `'payments'`

In [4]:
films = create_dataframe(user='tanya25pm', password='klcemshu', table_name='films')

payments = create_dataframe(user='tanya25pm', password='klcemshu', table_name='payments')

In [10]:
payments.dropna(subset='количество билетов', inplace=True) # удаление строк с пропусками (из конкретного столбца)

KeyError: ['количество билетов']

In [ ]:
payments.shape

(3962, 5)

In [ ]:
payments.isna()

,pay_date,film_id,part_of_day,client_id,count_tickets
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
3957,False,False,False,False,False
3958,False,False,False,False,False
3959,False,False,False,False,False
3960,False,False,False,False,False


In [ ]:
payments.duplicated().sum() # сумма дублей по всему датафрейму

11

In [ ]:
payments.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
3957    False
3958    False
3959    False
3960    False
3961    False
Length: 3962, dtype: bool

In [ ]:
payments[payments.duplicated()] # поиск дубликатов (полных)

,pay_date,film_id,part_of_day,client_id,count_tickets
3402,2024-03-20,48,утренний,YB14,1.0
3403,2024-03-20,48,утренний,YB14,1.0
3404,2024-03-04,49,утренний,JN23,1.0
3405,2024-04-20,42,вечерний,YH62,3.0
3406,2024-04-20,42,вечерний,YH62,3.0
3407,2024-03-10,37,дневной,QF27,5.0
3408,2024-04-20,42,вечерний,YH62,3.0
3409,2024-03-10,37,дневной,QF27,5.0
3410,2024-04-20,42,вечерний,YH62,3.0
3411,2024-03-10,37,дневной,QF27,5.0


In [ ]:
payments[payments.duplicated(subset=)] # поиск повторяющихся значений по столбцу

SyntaxError: invalid syntax (415264223.py, line 1)

In [40]:
payments.duplicated(subset='part_of_day')

0       False
1       False
2        True
3        True
4        True
        ...  
3957     True
3958     True
3959     True
3960     True
3961     True
Length: 3962, dtype: bool

In [ ]:
payments[payments.duplicated(keep=False)] # просмотр ВСЕХ полных дублей с помощью логической индексации 

,pay_date,film_id,part_of_day,client_id,count_tickets
141,2024-03-20,48,утренний,YB14,1.0
328,2024-03-04,49,утренний,JN23,1.0
397,2024-04-20,42,вечерний,YH62,3.0
490,2024-03-10,37,дневной,QF27,5.0
3402,2024-03-20,48,утренний,YB14,1.0
3403,2024-03-20,48,утренний,YB14,1.0
3404,2024-03-04,49,утренний,JN23,1.0
3405,2024-04-20,42,вечерний,YH62,3.0
3406,2024-04-20,42,вечерний,YH62,3.0
3407,2024-03-10,37,дневной,QF27,5.0


In [ ]:
payments.shape

(3962, 5)

In [42]:
payments = payments.drop_duplicates(keep='last') # удаление полных дублей (остается последняя (по порядку в датафрейме) строка)

In [41]:
payments.reset_index(drop=True, inplace=True) # сброс индекса (и удаление столбца со старым индексом)
payments

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0
2,2024-02-09,43,вечерний,XX54,5.0
3,2024-04-23,19,вечерний,QR90,3.0
4,2024-01-29,34,вечерний,SG93,5.0
...,...,...,...,...,...
3957,2024-02-11,1,дневной,FG89,4.0
3958,2024-04-08,34,утренний,DI23,2.0
3959,2024-01-16,10,дневной,ZJ39,4.0
3960,2024-04-15,28,вечерний,TU86,5.0


In [ ]:
payments \
    .drop_duplicates() \
    .reset_index(drop=True) \
    .shape # оформление нескольких методов

(3951, 5)

In [ ]:
payments['part_of_day'].unique() # список уникальных значений

array(['дневной', 'вечерний', 'утренний'], dtype=object)

In [ ]:
payments['film_id'].unique()

array([ 2, 15, 43, 19, 34, 13, 30, 21,  8, 31, 48, 22, 11, 46, 26, 45, 27,
        7, 10, 17, 42,  9, 24, 49, 35,  6, 23, 16,  1, 40,  4, 12, 47, 32,
       20, 18,  3, 28, 36, 33, 39, 14, 29, 25, 41,  5, 44, 37, 38, 53, 52,
       50], dtype=int64)

In [ ]:
payments['part_of_day'].nunique() # количество уникальных значений

3

In [ ]:
payments['film_id'].nunique()

52

In [ ]:
sorted(films.filmmaker.unique())

['Акира Куросава',
 'Альфред Хичкок',
 'Брайан Сингер',
 'Дамьен Шазель',
 'Дени Вильнёв',
 'Джеймс Кэмерон',
 'Джонатан Демми',
 'Джордж Лукас',
 'Джузеппе Торнаторе',
 'Дэвид Финчер',
 'Жерар Пирес',
 'Жуакин Душ Сантуш[англ.], Кэмп Пауэрс[англ.], Джастин Томпсон',
 'Ирвин Кершнер',
 'Исао Такахата',
 'Квентин Тарантино',
 'Кристофер Нолан',
 'Люк Бессон',
 'Майкл Кёртиц',
 'Мартин Скорсезе',
 'Масаки Кобаяси',
 'Милош Форман',
 'Оливье Накаш, Эрик Толедано',
 'Питер Джексон',
 'Пон Чжун Хо',
 'Ридли Скотт',
 'Роберт Земекис',
 'Роберт Земекис ',
 'Роберто Бениньи',
 'Роджер Аллерс, Роб Минкофф',
 'Роман Полански',
 'Серджо Леоне',
 'Сидни Люмет',
 'Стивен Спилберг',
 'Стивн Спилберг',
 'Тони Кэй',
 'Фернанду Мейреллиш',
 'Фрэнк Дарабонт',
 'Фрэнк Капра',
 'Фрэнсис Форд Коппола',
 'Хаяо Миядзаки',
 'Энди и Ларри Вачовски',
 'стивен спилберг']

In [ ]:
films.filmmaker.replace('стивен спилберг', 'Стивен Спилберг', inplace=True) # замена значений в ячейках

In [ ]:
sorted(films.filmmaker.unique())

['Акира Куросава',
 'Альфред Хичкок',
 'Брайан Сингер',
 'Дамьен Шазель',
 'Дени Вильнёв',
 'Джеймс Кэмерон',
 'Джонатан Демми',
 'Джордж Лукас',
 'Джузеппе Торнаторе',
 'Дэвид Финчер',
 'Жерар Пирес',
 'Жуакин Душ Сантуш[англ.], Кэмп Пауэрс[англ.], Джастин Томпсон',
 'Ирвин Кершнер',
 'Исао Такахата',
 'Квентин Тарантино',
 'Кристофер Нолан',
 'Люк Бессон',
 'Майкл Кёртиц',
 'Мартин Скорсезе',
 'Масаки Кобаяси',
 'Милош Форман',
 'Оливье Накаш, Эрик Толедано',
 'Питер Джексон',
 'Пон Чжун Хо',
 'Ридли Скотт',
 'Роберт Земекис',
 'Роберт Земекис ',
 'Роберто Бениньи',
 'Роджер Аллерс, Роб Минкофф',
 'Роман Полански',
 'Серджо Леоне',
 'Сидни Люмет',
 'Стивен Спилберг',
 'Стивн Спилберг',
 'Тони Кэй',
 'Фернанду Мейреллиш',
 'Фрэнк Дарабонт',
 'Фрэнк Капра',
 'Фрэнсис Форд Коппола',
 'Хаяо Миядзаки',
 'Энди и Ларри Вачовски']

In [ ]:
films.filmmaker = films.filmmaker \
    .replace('Стивн Спилберг', 'Стивен Спилберг') \
    .replace('Роберт Земекис ', 'Роберт Земекис')

In [ ]:
sorted(films.filmmaker.unique())

['Акира Куросава',
 'Альфред Хичкок',
 'Брайан Сингер',
 'Дамьен Шазель',
 'Дени Вильнёв',
 'Джеймс Кэмерон',
 'Джонатан Демми',
 'Джордж Лукас',
 'Джузеппе Торнаторе',
 'Дэвид Финчер',
 'Жерар Пирес',
 'Жуакин Душ Сантуш[англ.], Кэмп Пауэрс[англ.], Джастин Томпсон',
 'Ирвин Кершнер',
 'Исао Такахата',
 'Квентин Тарантино',
 'Кристофер Нолан',
 'Люк Бессон',
 'Майкл Кёртиц',
 'Мартин Скорсезе',
 'Масаки Кобаяси',
 'Милош Форман',
 'Оливье Накаш, Эрик Толедано',
 'Питер Джексон',
 'Пон Чжун Хо',
 'Ридли Скотт',
 'Роберт Земекис',
 'Роберто Бениньи',
 'Роджер Аллерс, Роб Минкофф',
 'Роман Полански',
 'Серджо Леоне',
 'Сидни Люмет',
 'Стивен Спилберг',
 'Тони Кэй',
 'Фернанду Мейреллиш',
 'Фрэнк Дарабонт',
 'Фрэнк Капра',
 'Фрэнсис Форд Коппола',
 'Хаяо Миядзаки',
 'Энди и Ларри Вачовски']

In [ ]:
films.release_year.sort_values(ascending=False) # сортировка сириес

20    2024
35    2023
33    2019
19    2014
41    2014
54    2012
52    2012
47    2011
13    2010
2     2008
42    2006
39    2006
6     2003
11    2002
32    2002
25    2002
8     2001
31    2001
36    2000
53    1999
27    1999
15    1999
12    1999
50    1998
24    1998
40    1998
26    1997
18    1995
45    1995
38    1994
7     1994
10    1994
37    1994
0     1994
5     1993
23    1991
28    1991
16    1990
51    1989
43    1988
48    1988
30    1985
14    1980
29    1977
17    1975
49    1975
3     1974
1     1972
9     1966
44    1962
34    1960
4     1957
22    1954
21    1946
46    1942
Name: release_year, dtype: int64

In [12]:
films.release_year.max()

2024

In [ ]:
films[films.release_year == films.release_year.max()] # вывод строки с самым новым фильмом из всего датафрейма

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
20,21,Дюна: Часть вторая,190,2024,Дени Вильнёв,"боевик, приключение, драма"


In [ ]:
films[films.price_ticket > films.price_ticket.mean()] # вывод фильмов с ценой билета выше среднего

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"
9,10,"Хороший, плохой, злой",450,1966,Серджо Леоне,"приключение, вестерн"
17,18,Пролетая над гнездом кукушки,340,1975,Милош Форман,драма
21,22,Эта прекрасная жизнь,340,1946,Фрэнк Капра,"драма, семейный фильм, фэнтези"
22,23,Семь самураев,450,1954,Акира Куросава,"боевик, драма"
23,24,Молчание ягнят,450,1991,Джонатан Демми,"детектив, драма, триллер"
24,25,Спасти рядового Райана,340,1998,Стивен Спилберг,"драма, военный фильм"
27,28,Зелёная миля,450,1999,Фрэнк Дарабонт,"детектив, драма, фэнтези"
30,31,Назад в будущее,340,1985,Роберт Земекис,"приключение, кинокомедия, научная фантастика"
31,32,Унесённые призраками,340,2001,Хаяо Миядзаки,"мультфильм, приключение, семейный фильм"


In [ ]:
# вывод количество с ценой билета ниже среднего

unique_films_cheap = films[films.price_ticket < films.price_ticket.mean()]['film_name'].nunique()
print(f'Количество дешевых фильмов: {unique_films_cheap}') 

Количество дешевых фильмов: 34


In [ ]:
payments.part_of_day.value_counts() # подсчет количества вида сеансов

part_of_day
вечерний    1349
утренний    1320
дневной     1293
Name: count, dtype: int64

In [ ]:
films.value_counts(subset='filmmaker', sort=True, normalize=True) # подсчет доли количества режиссеров

filmmaker
Стивен Спилберг                                                  0.090909
Кристофер Нолан                                                  0.072727
Питер Джексон                                                    0.054545
Роберт Земекис                                                   0.054545
Мартин Скорсезе                                                  0.036364
Дэвид Финчер                                                     0.036364
Энди и Ларри Вачовски                                            0.036364
Фрэнк Дарабонт                                                   0.036364
Фрэнсис Форд Коппола                                             0.036364
Тони Кэй                                                         0.018182
Фернанду Мейреллиш                                               0.018182
Оливье Накаш, Эрик Толедано                                      0.018182
Серджо Леоне                                                     0.018182
Роман Полански              

In [ ]:
payments.part_of_day.value_counts()

part_of_day
вечерний    1345
утренний    1317
дневной     1289
Name: count, dtype: int64

In [ ]:
payments.count_tickets

0       1.0
1       3.0
2       5.0
3       3.0
4       5.0
       ... 
3946    4.0
3947    2.0
3948    4.0
3949    5.0
3950    4.0
Name: count_tickets, Length: 3951, dtype: float64

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3927 non-null   object 
 1   film_id        3951 non-null   int64  
 2   part_of_day    3951 non-null   object 
 3   client_id      3951 non-null   object 
 4   count_tickets  3932 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 154.5+ KB


In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3927 non-null   object 
 1   film_id        3951 non-null   int64  
 2   part_of_day    3951 non-null   object 
 3   client_id      3951 non-null   object 
 4   count_tickets  3932 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 154.5+ KB


Важно! Аргумент `errors`. Принимает или `raise` (по умолчанию) - и тогда вызывает ошибку в случае, если не может перевести какие-то значения или `coerce` - тогда все значения, которые вызывает ошибку становятся NaN (типа float)

In [ ]:
payments.count_tickets = pd.to_numeric(payments.count_tickets, errors='coerce') # приведение к числовому типу

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3927 non-null   object 
 1   film_id        3951 non-null   int64  
 2   part_of_day    3951 non-null   object 
 3   client_id      3951 non-null   object 
 4   count_tickets  3932 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 154.5+ KB


In [ ]:
payments.film_id = payments.film_id.astype('str') # приведение к текстовому формату

In [ ]:
payments.film_id = payments.film_id.astype('int') # приведение к числовому формату

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3927 non-null   object 
 1   film_id        3951 non-null   int32  
 2   part_of_day    3951 non-null   object 
 3   client_id      3951 non-null   object 
 4   count_tickets  3932 non-null   float64
dtypes: float64(1), int32(1), object(3)
memory usage: 139.0+ KB


In [ ]:
payments.pay_date = pd.to_datetime(payments.pay_date)

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   pay_date       3927 non-null   datetime64[ns]
 1   film_id        3951 non-null   int32         
 2   part_of_day    3951 non-null   object        
 3   client_id      3951 non-null   object        
 4   count_tickets  3932 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 139.0+ KB


In [ ]:
payments.pay_date.astype('datetime64[ns]')

0      2024-04-24
1      2024-03-03
2      2024-02-09
3      2024-04-23
4      2024-01-29
          ...    
3946   2024-02-11
3947   2024-04-08
3948   2024-01-16
3949   2024-04-15
3950   2024-04-05
Name: pay_date, Length: 3951, dtype: datetime64[ns]

# Семинар 2

## Чтение JSON файла

In [5]:
pd.read_json('payments.json').info() #чтение json-файла

<class 'pandas.core.frame.DataFrame'>
Index: 3962 entries, 0 to 3961
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3938 non-null   float64
 1   film_id        3962 non-null   int64  
 2   part_of_day    3962 non-null   object 
 3   client_id      3962 non-null   object 
 4   count_tickets  3943 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 185.7+ KB


## Переименование всех столбцов

In [6]:
payments.columns

Index(['pay_date', 'film_id', 'part_of_day', 'client_id', 'count_tickets'], dtype='object')

In [ ]:
# новые названия столбцов
new_columns = ['дата_платежа', 'film_id', 'part_of_day', 'клиент', 'count_tickets']

In [7]:
payments.columns = new_columns

NameError: name 'new_columns' is not defined

In [8]:
payments.head()

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0
2,2024-02-09,43,вечерний,XX54,5.0
3,2024-04-23,19,вечерний,QR90,3.0
4,2024-01-29,34,вечерний,SG93,5.0


In [9]:
# вернем старые названия
old_columns = ['pay_date', 'film_id', 'part_of_day', 'client_id', 'count_tickets']

In [10]:
old_columns = ['pay_date', 'film_id', 'part_of_day', 'client_id', 'count_tickets']
payments.columns = old_columns

In [ ]:
payments.head()

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0
2,2024-02-09,43,вечерний,XX54,5.0
3,2024-04-23,19,вечерний,QR90,3.0
4,2024-01-29,34,вечерний,SG93,5.0


## Заполнение пропусков следующим значениями .ffill() и предыдущим значениями .bfill()

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3938 non-null   object 
 1   film_id        3962 non-null   int64  
 2   part_of_day    3962 non-null   object 
 3   client_id      3962 non-null   object 
 4   count_tickets  3943 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 154.9+ KB


In [ ]:
payments.count_tickets = payments.count_tickets.bfill() 

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_date       3938 non-null   object 
 1   film_id        3962 non-null   int64  
 2   part_of_day    3962 non-null   object 
 3   client_id      3962 non-null   object 
 4   count_tickets  3962 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 154.9+ KB


## Вставка столбца на конкретное место 

In [ ]:
payments.head()

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0
2,2024-02-09,43,вечерний,XX54,5.0
3,2024-04-23,19,вечерний,QR90,3.0
4,2024-01-29,34,вечерний,SG93,5.0


In [ ]:
payments.insert(2, 'скидка', 10)

In [ ]:
payments.head()

,дата_платежа,film_id,скидка,part_of_day,клиент,count_tickets
0,2024-04-24,2,10,дневной,NL42,1.0
1,2024-03-03,15,10,вечерний,HW75,3.0
2,2024-02-09,43,10,вечерний,XX54,5.0
3,2024-04-23,19,10,вечерний,QR90,3.0
4,2024-01-29,34,10,вечерний,SG93,5.0


In [ ]:
del payments['скидка']

In [ ]:
payments.head()

,pay_date,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,2,дневной,NL42,1.0
1,2024-03-03,15,вечерний,HW75,3.0
2,2024-02-09,43,вечерний,XX54,5.0
3,2024-04-23,19,вечерний,QR90,3.0
4,2024-01-29,34,вечерний,SG93,5.0


## Приведение к `datetime`

In [ ]:
payments.дата_платежа = pd.to_datetime(payments['pay_date'])

In [ ]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   дата_платежа   3927 non-null   datetime64[ns]
 1   film_id        3951 non-null   int32         
 2   part_of_day    3951 non-null   object        
 3   клиент         3951 non-null   object        
 4   count_tickets  3951 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int32(1), object(2)
memory usage: 139.0+ KB


## Углубление в groupby

In [ ]:
# Возьмем небольшой датафейм по одному клиент
client_YN41 = payments[payments['client_id']=='YN41']

client_YN41['pay_date'] = pd.to_datetime(client_YN41['pay_date'])
client_YN41['pay_month'] = client_YN41['pay_date'].dt.month
client_YN41

,pay_date,film_id,part_of_day,client_id,count_tickets,pay_month
352,2024-03-21,21,утренний,YN41,3.0,3.0
611,2024-03-03,46,вечерний,YN41,5.0,3.0
641,2024-04-26,25,дневной,YN41,5.0,4.0
693,2024-04-27,31,вечерний,YN41,2.0,4.0
1018,2024-04-06,19,дневной,YN41,1.0,4.0
1369,2024-04-04,22,вечерний,YN41,1.0,4.0
1405,2024-02-02,6,вечерний,YN41,5.0,2.0
1424,2024-02-02,45,дневной,YN41,4.0,2.0
1630,2024-04-27,45,дневной,YN41,5.0,4.0
1813,2024-04-02,38,вечерний,YN41,2.0,4.0


In [ ]:
# Сумма билетов по каждому part_of_day
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].sum()

,part_of_day,count_tickets
0,вечерний,20.0
1,дневной,24.0
2,утренний,20.0


In [ ]:
# Среднее значение
client_YN41.groupby(['part_of_day','pay_month'], as_index=False)[['film_id','count_tickets']].mean()

,part_of_day,pay_month,film_id,count_tickets
0,вечерний,1.0,35.000000,3.000000
1,вечерний,2.0,7.500000,3.500000
2,вечерний,3.0,46.000000,5.000000
3,вечерний,4.0,30.333333,1.666667
4,дневной,2.0,45.000000,4.000000
5,дневной,4.0,24.750000,3.250000
6,дневной,5.0,16.000000,4.000000
7,утренний,1.0,22.000000,5.000000
8,утренний,2.0,36.000000,5.000000
9,утренний,3.0,25.000000,2.333333


In [ ]:
# Медиана
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].median()

,part_of_day,count_tickets
0,вечерний,2.0
1,дневной,4.0
2,утренний,3.0


In [ ]:
# считает кол-во, но не учитывает null
client_YN41.groupby(by='part_of_day', as_index=False)['pay_date'].count()

,part_of_day,pay_date
0,вечерний,7
1,дневной,6
2,утренний,5


In [ ]:
# считает кол-во, но учитывает null
client_YN41.groupby(by='part_of_day', as_index=False)['pay_date'].size()

,part_of_day,size
0,вечерний,7
1,дневной,7
2,утренний,6


In [ ]:
# Минимиум 
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].min()

,part_of_day,count_tickets
0,вечерний,1.0
1,дневной,1.0
2,утренний,2.0


In [ ]:
# Максимум
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].max()

,part_of_day,count_tickets
0,вечерний,5.0
1,дневной,5.0
2,утренний,5.0


In [ ]:
client_YN41['pay_date'] = pd.to_datetime(client_YN41['pay_date'])
client_YN41 = client_YN41.sort_values('pay_date')
client_YN41

,pay_date,film_id,part_of_day,client_id,count_tickets,pay_month
2540,2024-01-01,35,вечерний,YN41,3.0,1.0
2376,2024-01-23,22,утренний,YN41,5.0,1.0
1405,2024-02-02,6,вечерний,YN41,5.0,2.0
1424,2024-02-02,45,дневной,YN41,4.0,2.0
2653,2024-02-14,36,утренний,YN41,5.0,2.0
2683,2024-02-19,9,вечерний,YN41,2.0,2.0
611,2024-03-03,46,вечерний,YN41,5.0,3.0
3207,2024-03-13,6,утренний,YN41,2.0,3.0
352,2024-03-21,21,утренний,YN41,3.0,3.0
2622,2024-03-26,48,утренний,YN41,2.0,3.0


In [ ]:
# первое значение для группировки
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].first()

,part_of_day,count_tickets
0,вечерний,3.0
1,дневной,4.0
2,утренний,5.0


In [ ]:
# последнее значение для группировки
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].last()

,part_of_day,count_tickets
0,вечерний,2.0
1,дневной,3.0
2,утренний,3.0


In [ ]:
# Описание всех важных статистик
client_YN41.groupby(by='part_of_day', as_index=False)['count_tickets'].describe()

,part_of_day,count,mean,std,min,25%,50%,75%,max
0,вечерний,7.0,2.857143,1.573592,1.0,2.00,2.0,4.0,5.0
1,дневной,7.0,3.428571,1.511858,1.0,2.50,4.0,4.5,5.0
2,утренний,6.0,3.333333,1.366260,2.0,2.25,3.0,4.5,5.0


In [ ]:
# Если нужно по-раному посчитать. Например: для одного столбца mean, для другого min, а для другого size
df_result = client_YN41.groupby('part_of_day', as_index=False).agg(
    {
        'count_tickets':'mean',
        'pay_month':'min',
        'client_id':'size'
    })

df_result.columns =['Время суток', 'Среднее кол-во купленных билетов', 'Первый месяц покупки', 'Кол-во покупок']
df_result

,Время суток,Среднее кол-во купленных билетов,Первый месяц покупки,Кол-во покупок
0,вечерний,2.857143,1.0,7
1,дневной,3.428571,2.0,7
2,утренний,3.333333,1.0,6


# Домашнее задание по семинару 👨‍💻

In [11]:
#payments = create_dataframe(user='xxxxx', password='xxxxx', table_name='payments')
payments = create_dataframe(user='tanya25pm', password='klcemshu', table_name='payments')


#### 1. Заполните пропуски в столбце `pay_date` следующим значениями

In [13]:
# your code 
payments['pay_date'] = payments['pay_date'].ffill() 

#### 2. Создайте столбец `pay_month` с номеров месяца на основе `pay_date`. Пусть этот столбец будет после `pay_date`

In [14]:
# your code 
payments['pay_date'] = pd.to_datetime(payments['pay_date'])

payments.insert(1, 'pay_month', payments['pay_date'].dt.month)
payments.head(3)


,pay_date,pay_month,film_id,part_of_day,client_id,count_tickets
0,2024-04-24,4,2,дневной,NL42,1.0
1,2024-03-03,3,15,вечерний,HW75,3.0
2,2024-02-09,2,43,вечерний,XX54,5.0


#### 3. Посчитайте по каждому месяцу:
- сумму всех билетов
- кол-во клиентов

Переименуйте столбцы на русский язык с понятным описанием

In [16]:
# your code считаем сумму всех билетов

payments.rename({'count_tickets': 'количество_билетов', 'pay_month': 'месяц_оплаты', 'client_id': 'номер_клиента' },inplace=True, axis=1) 
payments.groupby(by='месяц_оплаты', as_index=False)['количество_билетов'].sum()

,месяц_оплаты,количество_билетов
0,1,2923.0
1,2,2741.0
2,3,3059.0
3,4,2924.0
4,5,99.0
5,7,5.0


In [20]:
# your code считаем количество клиентов
clients = payments.groupby(by='месяц_оплаты', as_index=False)['номер_клиента'].nunique()
clients.rename({ 'номер_клиента': 'количество_клиентов' },inplace=True, axis=1) 
display(clients)

,месяц_оплаты,количество_клиентов
0,1,198
1,2,197
2,3,197
3,4,199
4,5,37
5,7,1
